### Specify Inputs

In [25]:
# This is the experiment code we want to look at.
search = "MLB000|MLB003|QSC022|MLB006|MLD023" #for all 6L baseline cells
search = "MLB006|QSC022|MLD023A" #for 6L baseline cells that are still cycling
#search = "MLD012|MLD016|MLD018" #for 22L cells
#search = "MLD023" #for 6L builds from Cathy cells


GroupBy = 'Experiment'  #Group cells by Experiment code (MLB006, MLD023, FCD00, etc) in screen and cycle plots
#GroupBy = 'WorkWeek'  #Group cells by work week built in screen and cycle plots
#GroupBy = 'Batch'   #Group cells by batches (MLB006AA, MLB006AB, MLB00AC, etc) in screen and cycle plots


CellsToExclude = "MLD023AB-PS00-01|MLD023AB-PS00-02|MLD023AB-PS00-05|QSC022AG-PS00-03" #specify any cells that you do not want to include in BOTH yield and reliability cycling
CellsThatPassedScreen = ['MLD023AC-PS00-02','MLD023AD-PS00-01','MLD023AD-PS00-02','MLD023AD-PS00-03', 'MLD023AD-PS00-04', 'MLD023AD-PS00-05'] # List of cells that did not fail screen despite datahub saying it did
CellsThatFailedScreen = ['MLD023AF-PS00-03'] # List of cells that should have failed screen despite datahub saying it didn't (i.e. "soft-shorted" during screening)

builtby = "7/01/2024" #search for cells built on or after this date


### Import Functions and Modules

In [26]:
#Import libraries and functions
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter
from lifelines.utils import restricted_mean_survival_time
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.io as pio
import plotly.graph_objects as go
from qsdc.client import Client
from datetime import datetime
import warnings


pio.templates.default = "plotly_white"
clrs = plotly.colors.DEFAULT_PLOTLY_COLORS
qs_client = Client()

#Remove warning messages
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)
# Convert builtby to datetime
builtby_date = pd.to_datetime(builtby, format="%m/%d/%Y")

### Pull from Datahub


In [27]:
#Pull multilayer screen data
MLScreenEach = qs_client.data_hub.get_dataset(dataset = 'MFG-80L-6L-SCREEN-CYCLE') ##electrical test data of unit cells for each cycle
MLScreenSummary = qs_client.data_hub.get_dataset(dataset = 'MFG-80L-6L-SCREEN') ##electrical test data of unit cells that sumamrizes each screen test
#Pull multilayer track cycle data
MLTrackCycleEach = qs_client.data_hub.get_dataset(dataset = 'MFG-80L-TRACK-CYCLE-REL-CYCLE')
MLTrackCycleSummary = qs_client.data_hub.get_dataset(dataset = 'MFG-80L-TRACK-CYCLE-REL')
#Pull geneology/multilayer info of unit cells
dmlg = qs_client.data_hub.get_dataset(dataset = 'MFG-80L-6L-PRODUCTION') ##multilayer info (ML6_id)
dmlg = dmlg.drop_duplicates(subset='US_id')

MLSlct= qs_client.data_hub.get_dataset(dataset = 'MFG-80L-6L-SLCT-CYCLE')

### Create Master Spreadsheets

In [28]:
##Create a Master Spreadsheet for Screening data
# Grab electrical metrics by consolidating individual cycles in screen
def last_non_missing(series):
    return series.dropna().iloc[-1] if not series.dropna().empty else None
def minimum_non_missing(series):
    return series.dropna().min() if not series.dropna().empty else None
# Group by cell id and consolidate
dfc_consolidated = (
    MLScreenEach.groupby('ML6T_id', as_index=False)
    .agg({
        'AMSDcCapacity': last_non_missing,
        'DischargeCapacity': last_non_missing,
        'dvdt':minimum_non_missing, 
        'MedDcASR': last_non_missing
    })
)
dfc_consolidated = dfc_consolidated.rename(columns={
    'AMSDcCapacity': 'CellSpecificDischargeCapacity',
    'DischargeCapacity': 'CellCo3DischargeCapcity',
    'dvdt': 'Celldvdt',
    'MedDcASR': 'Cell1CDischargeASR'
})
MLScreenSummary = MLScreenSummary.merge(
    dfc_consolidated,
    left_on='ML6T_id',
    right_on='ML6T_id',
    how='inner'  # Use 'inner' to keep only matching rows; adjust to 'left' or 'outer' as needed
)

#Filter by what we are searching for
filtered_MLScreenSummary = MLScreenSummary[MLScreenSummary['ML6T_id'].str.contains(search, regex=True)]
filtered_MLScreenEach = MLScreenEach[MLScreenEach['ML6T_id'].str.contains(search, regex=True)]
#Overwrite screen yield for cells that didn't really short
columns_to_overwrite = ['stage1_yield', 'stage2_yield', 'stage3_yield', 'stage4_yield', 'stage5_yield'] # Specify columns to overwrite
filtered_MLScreenSummary.loc[filtered_MLScreenSummary['ML6T_id'].isin(CellsThatPassedScreen), columns_to_overwrite] = 1 # Overwrite the specified columns with 1 for matching rows
# Suppose CellsThatFailedScreen is a list of strings
filtered_MLScreenSummary.loc[filtered_MLScreenSummary['ML6T_id'].isin(CellsThatFailedScreen), columns_to_overwrite] = 0 # Overwrite the specified columns with 1 for matching rows

columns_to_overwrite2 = ['AnyFailure', 'CycleFailure'] # Specify columns to overwrite
filtered_MLScreenSummary.loc[filtered_MLScreenSummary['ML6T_id'].isin(CellsThatPassedScreen), columns_to_overwrite2] = False

# Exclude cells
MLScreenMaster = filtered_MLScreenSummary[~filtered_MLScreenSummary['ML6T_id'].str.contains(CellsToExclude)]

# Add additional columns that are typically used for grouping
MLScreenMaster['Experiment']= MLScreenMaster['ML6T_id'].str[:6]  #Group by experiment
MLScreenMaster['Batch']= MLScreenMaster['ML6T_id'].str[:8]  #Group by experiment
MLScreenMaster['WorkWeek']= MLScreenMaster['TestCycleStart_week_first']  #Group by experiment


# Group screen yield by:
MLScreenMaster['Group']= MLScreenMaster[GroupBy]  #Group by experiment



In [29]:
##Create a Master Spreadsheets for Cycling Data

#Filter by what we are searching for
MLTrackCycleSummary = MLTrackCycleSummary[MLTrackCycleSummary['sample_id'].str.contains(search, regex=True)]
MLTrackCycleEach = MLTrackCycleEach[MLTrackCycleEach['sample_id'].str.contains(search, regex=True)]

# Exclude cells
MLTrackCycleSummary = MLTrackCycleSummary[~MLTrackCycleSummary['sample_id'].str.contains(CellsToExclude)]
MLTrackCycleEach = MLTrackCycleEach[~MLTrackCycleEach['sample_id'].str.contains(CellsToExclude)]

non_empty_failures = [cell for cell in CellsThatFailedScreen if cell.strip()]
if non_empty_failures:
    pattern = '|'.join(non_empty_failures)
    MLTrackCycleSummary = MLTrackCycleSummary[~MLTrackCycleSummary['sample_id'].str.contains(pattern)]
    MLTrackCycleEach = MLTrackCycleEach[~MLTrackCycleEach['sample_id'].str.contains(pattern)]


# Add additional columns that are typically used for grouping
MLTrackCycleEach['Experiment']= MLTrackCycleEach['sample_id'].str[:6]  #Group by experiment
MLTrackCycleEach['Batch']= MLTrackCycleEach['sample_id'].str[:8]  #Group by experiment
MLTrackCycleEach['WorkWeek']= MLTrackCycleEach['TestCycleStart_week_first']  #Group by experiment


# Group screen yield by:
MLTrackCycleEach['Group']= MLTrackCycleEach[GroupBy]  #Group by experiment


In [30]:
##Tier Each ML Pouch
#Pull geneology/multilayer info
dmlg = qs_client.data_hub.get_dataset(dataset = 'MFG-80L-6L-PRODUCTION') ##multilayer info (ML6_id)
dmlg = dmlg.drop_duplicates(subset='US_id')
CellsInML = MLScreenMaster.merge(dmlg, left_on='ML6T_id', right_on='ML6_id', how='left')
CellsInML = CellsInML[['ML6T_id', 'US_id']].rename(columns={'US_id': 'Cell ID'})

#Pull cell metrology data from datahub, both standard/auto metrology and manual review
dfctq = qs_client.data_hub.get_dataset(dataset = 'MFG-60L-UC-CTQ') ## standard metro
dfctq_filtered = dfctq[dfctq['US_id'].isin(CellsInML['Cell ID'])]
yielded_dfctq = dfctq_filtered[dfctq_filtered['unit_cell_test_yield'] == 1] #keep cells that yielded

dfmr = qs_client.data_hub.get_dataset(dataset = 'MFG-60L-UC-MR') ## manual review
dfmr_filtered = dfmr[dfmr['US_id'].isin(CellsInML['Cell ID'])]
yielded_dfmr = dfmr_filtered[dfmr_filtered['unit_cell_test_yield'] == 1] #keep cells that yielded


# First, merge the DataFrames on 'US_id' to align rows
merged_df = yielded_dfctq.merge(yielded_dfmr[['US_id', 'edge_thickness_tier_us_mr', 'A1_anode_tier_top_us_mr', 'A1_anode_tier_bottom_us_mr',
                                              'cathode_alignment_custom_model_tier_us_mr', 'median_contour_catholyte_pct_us_mr', 'disposition_mr', 'failure_modes_mr']], on='US_id', how='left')
# Then, overwrite 'edge_thickness_tier_us' in 'filtered_dfctq' where 'edge_thickness_tier_us_mr' has a value
merged_df['edge_thickness_tier_us'] = merged_df['edge_thickness_tier_us_mr'].combine_first(merged_df['edge_thickness_tier_us'])
# Then, overwrite 'A1_anode_tier_top_us' in 'filtered_dfctq' where 'A1_anode_tier_top_us_mr' has a value
merged_df['A1_anode_tier_top_us'] = merged_df['A1_anode_tier_top_us_mr'].combine_first(merged_df['A1_anode_tier_top_us'])
# Then, overwrite 'A1_anode_tier_bottom_us' in 'filtered_dfctq' where 'A1_anode_tier_bottom_us_mr' has a value
merged_df['A1_anode_tier_bottom_us'] = merged_df['A1_anode_tier_bottom_us_mr'].combine_first(merged_df['A1_anode_tier_bottom_us'])
# Then, overwrite 'A1_anode_tier_bottom_us' in 'filtered_dfctq' where 'A1_anode_tier_bottom_us_mr' has a value
merged_df['cathode_alignment_custom_model_tier_us'] = merged_df['cathode_alignment_custom_model_tier_us_mr'].combine_first(merged_df['cathode_alignment_custom_model_tier_us'])
# Then, overwrite 'A1_anode_tier_bottom_us' in 'filtered_dfctq' where 'A1_anode_tier_bottom_us_mr' has a value
merged_df['median_contour_catholyte_pct_us'] = merged_df['median_contour_catholyte_pct_us_mr'].combine_first(merged_df['median_contour_catholyte_pct_us'])
# Then, overwrite 'disposition_us' in 'disposition_mr' has a value
merged_df['disposition'] = merged_df['disposition_mr'].combine_first(merged_df['disposition'])
# Then, overwrite 'disposition_us' in 'disposition_mr' has a value
merged_df['failure_modes'] = merged_df['failure_modes_mr'].combine_first(merged_df['failure_modes'])
# Drop the 'edge_thickness_tier_us_mr' column if you don't need it
dfctq_updated = merged_df.drop(columns=['edge_thickness_tier_us_mr', 'A1_anode_tier_top_us_mr', 'A1_anode_tier_bottom_us_mr',
                                              'cathode_alignment_custom_model_tier_us_mr', 'median_contour_catholyte_pct_us_mr', 'disposition_mr','failure_modes_mr' ])

#Update Final Tier of Cells
conditions = [
    dfctq_updated['disposition'] == 'Tier 1',
    dfctq_updated['disposition'] == 'Tier 2',
    dfctq_updated['disposition'] == 'Fail',
    dfctq_updated['disposition'] == 'Scrap',
    dfctq_updated['disposition'] == 'Missing Data',
]
choices = ['1', '2', '3','Scrapped', 'TBD']
dfctq_updated['Tier'] = np.select(conditions, choices)



# Merge dfctq_updated['US_id', 'Tier'] with CellsInML based on 'US_id'
CellsInML = CellsInML.merge(dfctq_updated[['US_id', 'Tier']], 
                            left_on='Cell ID', 
                            right_on='US_id', 
                            how='left')


# Assign the 'Tier' column to 'Cell Tier' and drop the extra 'US_id' column
CellsInML['Cell Tier'] = CellsInML['Tier']
CellsInML = CellsInML.drop(columns=['Tier', 'US_id'])

# Group by "samplename" and find the max "Cell Tier" for each
CellsInML['Cell Tier'] = pd.to_numeric(CellsInML['Cell Tier'], errors='coerce')
FinalMLTier = CellsInML.groupby('ML6T_id', as_index=False)['Cell Tier'].max()


# Rename columns as required
FinalMLTier.columns = ['Multilayer', 'ML Tier']
# Convert "ML Tier" to integer and format as "Tier {max Cell Tier}"
# Conditionally update 'ML Tier'
FinalMLTier['ML Tier'] = np.where(
    FinalMLTier['ML Tier'].isna(), 
    np.nan,  # Keep NaN if it was originally NaN
    "Tier " + FinalMLTier['ML Tier'].fillna(0).astype(int).astype(str)
)

# Merge df_master with FinalMLTier on "samplename" and "Multilayer"
MLScreenMaster= MLScreenMaster.merge(FinalMLTier, left_on='ML6T_id', right_on='Multilayer', how='left')

# Update "cell_tier_group" with the values from "ML Tier"
MLScreenMaster['cell_tier_group'] = MLScreenMaster['ML Tier']

# Drop the extra "Multilayer" and "ML Tier" columns
MLScreenMaster = MLScreenMaster.drop(columns=['Multilayer', 'ML Tier'])

### Plot Multilayer Screen Yield

In [31]:
#Plot Screen Yield
finishedScreen = MLScreenMaster[MLScreenMaster["stage4_finished"] == 1]  # Ensure it's an integer, not a string
grouped = finishedScreen.groupby('Group')


# Calculate normalized bar values for each group
bar_data = []
group_labels = []
for group, df_group in grouped:
    total_rows = len(df_group)
    stage1_count = df_group['stage1_yield'].sum()
    stage2_count = df_group['stage2_yield'].sum()
    stage3_count = df_group['stage3_yield'].sum()
    stage5_count = df_group['stage4_yield'].sum()
    
    # Normalize the values to percentages
    bar_values_percentage = [
        (total_rows / total_rows) * 100,                          # Total rows (always 100%)
        (stage1_count / total_rows) * 100,         # Excluding stage1
        (stage2_count / total_rows) * 100,  # Excluding stage2
        (stage3_count / total_rows) * 100,  # Excluding stage3
        (stage5_count / total_rows) * 100                         # Stage5 (and Stage4) count
    ]
    
    bar_data.append(bar_values_percentage)
    group_labels.append(group)

# Define bar labels
bar_labels = ['Cells Built', 'Initial 1C Yield', 'Fast-Charge Yield', 'Final 1C Yield', 'C/3 Yield']
# Create a new DataFrame for Plotly Express
plot_data = pd.DataFrame(bar_data, columns=bar_labels, index=group_labels)
# Reset index for x-axis labeling
plot_data.reset_index(inplace=True)
plot_data.rename(columns={'index': 'Group'}, inplace=True)

# Define colors for each bar
colors = [
    px.colors.qualitative.Plotly[2],  # Color for 'Total Rows'
    px.colors.qualitative.Plotly[5],  # Color for 'Excluding Stage1'
    px.colors.qualitative.Plotly[3],  # Color for 'Excluding Stage2'
    px.colors.qualitative.Plotly[6],  # Color for 'Excluding Stage3'
    px.colors.qualitative.Plotly[4]   # Color for 'Stage5 and Stage4 Count'
]
# Create the interactive bar chart with Plotly Express, specifying width, height, and custom font sizes
fig = px.bar(
    plot_data, 
    x='Group', 
    y=bar_labels, 
    title="Multilayer Screen Yield", 
    labels={'value': 'Percentage (%)'}, 
    barmode='group', 
    color_discrete_sequence=colors,
    width=1200,  # Adjust width
    height=700   # Adjust height
)

# Update layout to increase font sizes
fig.update_layout(
    title=dict(font=dict(size=24)),          # Title font size
    xaxis=dict(title=dict(font=dict(size=18)), tickfont=dict(size=16)),  # X-axis label and ticks
    yaxis=dict(title=dict(font=dict(size=18)), tickfont=dict(size=16)),  # Y-axis label and ticks
    legend=dict(font=dict(size=16)),         # Legend font size
)

# Add annotation for the total samples in the first bar (Cells Built)
for i, group in enumerate(plot_data['Group']):
    # Get the height of the first bar ('Cells Built') for the current group
    first_bar_value = plot_data.loc[i, 'Cells Built']
    total_samples = len(MLScreenMaster[MLScreenMaster['Group'] == group])
    fig.add_annotation(
        x=group, 
        y=first_bar_value,  # Place the label just above the first bar
        text=f"N={total_samples}", 
        showarrow=False, 
        font=dict(size=14, color="black"),  # Annotation font size
        align="center",
        xshift=-100, 
        yshift=10  # Position slightly above the bar
    )

    # Add annotation for the final percentage in the last bar (C/3 Yield)
    final_percentage = plot_data.loc[i, 'C/3 Yield']
    fig.add_annotation(
        x=group, 
        y=final_percentage, 
        text=f"{final_percentage:.1f}%", 
        showarrow=False, 
        font=dict(size=14, color="black"),  # Annotation font size
        align="center", 
        xshift = 100, 
        yshift=10  # Position slightly above the bar
    )
# add grey dotted line at 80% yield
fig.add_shape(
    type="line",
    x0=-0.5,
    x1=plot_data.shape[0] - 0.5,
    y0=80,
    y1=80,
    line=dict(color="grey", width=2, dash="dot"),
    )
# Show the plot
fig.show()

### Plot Cell Metrics

In [32]:
# Modify MLScreenMaster by removing non-yielded cells and cells we had to hardcode
YieldedMLScreenMaster = MLScreenMaster.loc[MLScreenMaster['stage5_yield'] == 1]  # Keep cells that yielded for these plots
YieldedMLScreenMaster = YieldedMLScreenMaster[~YieldedMLScreenMaster['ML6T_id'].isin(CellsThatPassedScreen)]  # Filter the DataFrame to exclude rows where 'ML6T_id' is in CellsThatPassedScreen
YieldedMLScreenMaster.reset_index(drop=True, inplace=True)  # Reset the index for the resulting DataFrame (optional)

# Assuming YieldedMLScreenMaster is already loaded and has the necessary columns
color_by = 'Group'  # Color differentiation by Group
grouping = 'Group'

# Add a new numeric column for x positions to shift points
YieldedMLScreenMaster['group_numeric'] = pd.factorize(YieldedMLScreenMaster[grouping])[0]

# Create a color dictionary for each unique value in Group
color = dict(zip(YieldedMLScreenMaster[color_by].unique(), px.colors.qualitative.Plotly * 5))

# Initialize subplot layout
fig = make_subplots(
    rows=1, cols=2,
    horizontal_spacing=0.12,
    vertical_spacing=0.1,
    shared_xaxes=True,
    subplot_titles=[
        "Specific Discharge Capacity",
        "Cell Co3 Discharge Capacity"
    ]
)

# Set a flag to ensure legend items are added only once
legend_added = {key: False for key in YieldedMLScreenMaster[color_by].unique()}

# Add traces for the first plot (CellSpecificDischargeCapacity)
for label, group in YieldedMLScreenMaster.groupby(grouping):
    for color_value, group_color in group.groupby(color_by):
        # Add box plot
        fig.add_trace(
            go.Box(
                x=group_color['group_numeric'],  # Use the numeric version of the grouping values
                y=group_color["CellSpecificDischargeCapacity"],
                quartilemethod="linear",
                name=color_value,
                text=group_color["sample_id"] if "sample_id" in group_color.columns else None,
                #showlegend=not legend_added[color_value],
                showlegend=False,
                fillcolor=color[color_value],
                line=dict(color="black"),
            ),
            row=1, col=1
        )

        # Add scatter points for individual data (with a slight shift to the left)
        fig.add_trace(
            go.Scatter(
                x=group_color['group_numeric'] - 0.35,  # Shift numeric positions to the left
                y=group_color["CellSpecificDischargeCapacity"],
                mode="markers",
                marker=dict(color=color[color_value], size=6, symbol='circle'),
                name=f"{color_value} Points",  # Separate legend for points
                legendgroup=color_value,  # Group the legend for boxes and points
                showlegend=False,  # Legend already handled for boxes
            ),
            row=1, col=1
        )
        
        legend_added[color_value] = True

# Reset the legend flags for the second plot
legend_added = {key: False for key in YieldedMLScreenMaster[color_by].unique()}

# Add traces for the second plot (CellCo3DischargeCapcity)
for label, group in YieldedMLScreenMaster.groupby(grouping):
    for color_value, group_color in group.groupby(color_by):
        # Add box plot
        fig.add_trace(
            go.Box(
                x=group_color['group_numeric'],  # Use the numeric version of the grouping values
                y=group_color["CellCo3DischargeCapcity"],
                quartilemethod="linear",
                name=color_value,
                text=group_color["sample_id"] if "sample_id" in group_color.columns else None,
                showlegend=not legend_added[color_value],
                fillcolor=color[color_value],
                line=dict(color="black"),
            ),
            row=1, col=2
        )

        # Add scatter points for individual data (with a slight shift to the left)
        fig.add_trace(
            go.Scatter(
                x=group_color['group_numeric'] - 0.35,  # Shift numeric positions to the left
                y=group_color["CellCo3DischargeCapcity"],
                mode="markers",
                marker=dict(color=color[color_value], size=6, symbol='circle'),
                name=f"{color_value} Points",  # Separate legend for points
                legendgroup=color_value,  # Group the legend for boxes and points
                showlegend=False,  # Legend already handled for boxes
            ),
            row=1, col=2
        )
        
        legend_added[color_value] = True

# Update y-axes titles
fig.update_yaxes(
    title_text="Discharge Capacity (mAh/g)",
    row=1, col=1
)
fig.update_yaxes(
    title_text="Co3 Discharge Capacity (mAh)",
    row=1, col=2
)

# Update x-axes titles
fig.update_xaxes(title_text="Group", row=1, col=1)
fig.update_xaxes(title_text="Group", row=1, col=2)

# Update x-axes ticks to show original grouping labels
fig.update_xaxes(
    tickmode='array',
    tickvals=YieldedMLScreenMaster['group_numeric'].unique(),
    ticktext=YieldedMLScreenMaster[grouping].unique(),
    row=1, col=1,
    ticks="outside",  # Add ticks outside the plot
)
fig.update_xaxes(
    tickmode='array',
    tickvals=YieldedMLScreenMaster['group_numeric'].unique(),
    ticktext=YieldedMLScreenMaster[grouping].unique(),
    row=1, col=2,
    ticks="outside",  # Add ticks outside the plot
)

# Update y-axes ticks to show original grouping labels
fig.update_yaxes(
    ticks="outside",  # Add ticks outside the plot
    row=1, col=1
)
fig.update_yaxes(
    ticks="outside",  # Add ticks outside the plot
    row=1, col=2
)

# Add border to both subplots and adjust layout
fig.update_layout(
    title_text="",
    height=600,
    width=1200,
    showlegend=True,
    xaxis=dict(showgrid=True, zeroline=True, showline=True, linecolor='black', linewidth=2, mirror=True, ticks="outside"),  # Border and ticks for the first subplot
    yaxis=dict(showgrid=True, zeroline=True, showline=True, linecolor='black', linewidth=2, mirror=True, ticks="outside"),  # Border and ticks for the first subplot
    plot_bgcolor='white',  # Set plot background color to white
    paper_bgcolor='white',  # Set paper background color to white
    xaxis2=dict(showgrid=True, zeroline=True, showline=True, linecolor='black', linewidth=2, mirror=True, ticks="outside"),  # Border and ticks for second subplot x-axis (all sides)
    yaxis2=dict(showgrid=True, zeroline=True, showline=True, linecolor='black', linewidth=2, mirror=True, ticks="outside"),  # Border and ticks for second subplot y-axis (all sides)
)

# Display the plot
fig.show()


In [33]:
# Initialize subplot layout
fig = make_subplots(
    rows=1, cols=2,
    horizontal_spacing=0.12,
    vertical_spacing=0.1,
    shared_xaxes=True,
    subplot_titles=[
        "dV/dt",
        "Discharge ASR"
    ]
)

# Set a flag to ensure legend items are added only once
legend_added = {key: False for key in YieldedMLScreenMaster[color_by].unique()}

# Add traces for the first plot (Celldvdt)
for label, group in YieldedMLScreenMaster.groupby(grouping):
    for color_value, group_color in group.groupby(color_by):
        # Add box plot
        fig.add_trace(
            go.Box(
                x=group_color['group_numeric'],  # Use the numeric version of the grouping values
                y=group_color["Celldvdt"],
                quartilemethod="linear",
                name=color_value,
                text=group_color["sample_id"] if "sample_id" in group_color.columns else None,
                #showlegend=not legend_added[color_value],
                showlegend=False,
                fillcolor=color[color_value],
                line=dict(color="black"),
            ),
            row=1, col=1
        )

        # Add scatter points for individual data (with a slight shift to the left)
        fig.add_trace(
            go.Scatter(
                x=group_color['group_numeric'] - 0.35,  # Shift numeric positions to the left
                y=group_color["Celldvdt"],
                mode="markers",
                marker=dict(color=color[color_value], size=6, symbol='circle'),
                name=f"{color_value} Points",  # Separate legend for points
                legendgroup=color_value,  # Group the legend for boxes and points
                showlegend=False,  # Legend already handled for boxes
            ),
            row=1, col=1
        )
        
        legend_added[color_value] = True

# Reset the legend flags for the second plot
legend_added = {key: False for key in YieldedMLScreenMaster[color_by].unique()}

# Add traces for the second plot (Cell1CDischargeASR)
for label, group in YieldedMLScreenMaster.groupby(grouping):
    for color_value, group_color in group.groupby(color_by):
        # Add box plot
        fig.add_trace(
            go.Box(
                x=group_color['group_numeric'],  # Use the numeric version of the grouping values
                y=group_color["Cell1CDischargeASR"],
                quartilemethod="linear",
                name=color_value,
                text=group_color["sample_id"] if "sample_id" in group_color.columns else None,
                showlegend=not legend_added[color_value],
                fillcolor=color[color_value],
                line=dict(color="black"),
            ),
            row=1, col=2
        )

        # Add scatter points for individual data (with a slight shift to the left)
        fig.add_trace(
            go.Scatter(
                x=group_color['group_numeric'] - 0.35,  # Shift numeric positions to the left
                y=group_color["Cell1CDischargeASR"],
                mode="markers",
                marker=dict(color=color[color_value], size=6, symbol='circle'),
                name=f"{color_value} Points",  # Separate legend for points
                legendgroup=color_value,  # Group the legend for boxes and points
                showlegend=False,  # Legend already handled for boxes
            ),
            row=1, col=2
        )
        
        legend_added[color_value] = True

# Update y-axes titles
fig.update_yaxes(
    title_text="dV/dt (µV/s)",
    row=1, col=1
)
fig.update_yaxes(
    title_text="1C Discharge ASR (Ohm cm<sup>2</sup>)",
    row=1, col=2
)

# Update x-axes titles
fig.update_xaxes(title_text="Group", row=1, col=1)
fig.update_xaxes(title_text="Group", row=1, col=2)

# Update x-axes ticks to show original grouping labels
fig.update_xaxes(
    tickmode='array',
    tickvals=YieldedMLScreenMaster['group_numeric'].unique(),
    ticktext=YieldedMLScreenMaster[grouping].unique(),
    row=1, col=1,
    ticks="outside",  # Add ticks outside the plot
)
fig.update_xaxes(
    tickmode='array',
    tickvals=YieldedMLScreenMaster['group_numeric'].unique(),
    ticktext=YieldedMLScreenMaster[grouping].unique(),
    row=1, col=2,
    ticks="outside",  # Add ticks outside the plot
)

# Update y-axes ticks to show original grouping labels
fig.update_yaxes(
    ticks="outside",  # Add ticks outside the plot
    row=1, col=1
)
fig.update_yaxes(
    ticks="outside",  # Add ticks outside the plot
    row=1, col=2
)

# Add border to both subplots and adjust layout
fig.update_layout(
    title_text="",
    height=600,
    width=1200,
    showlegend=True,
    xaxis=dict(showgrid=True, zeroline=True, showline=True, linecolor='black', linewidth=2, mirror=True, ticks="outside"),  # Border and ticks for the first subplot
    yaxis=dict(showgrid=True, zeroline=True, showline=True, linecolor='black', linewidth=2, mirror=True, ticks="outside"),  # Border and ticks for the first subplot
    plot_bgcolor='white',  # Set plot background color to white
    paper_bgcolor='white',  # Set paper background color to white
    xaxis2=dict(showgrid=True, zeroline=True, showline=True, linecolor='black', linewidth=2, mirror=True, ticks="outside"),  # Border and ticks for second subplot x-axis (all sides)
    yaxis2=dict(showgrid=True, zeroline=True, showline=True, linecolor='black', linewidth=2, mirror=True, ticks="outside"),  # Border and ticks for second subplot y-axis (all sides)
)

# Display the plot
fig.show()


### Plot Vmin Track Cycling

In [34]:
# Order dataframe so it appears nicely in Vmin Reliability Plot
MLTrackCycleEach = MLTrackCycleEach.sort_values(by=['sample_id', 'track_cycle_count_cumulative'])

# Create a scatter/line plot
fig = px.scatter(
    MLTrackCycleEach,
    x='track_cycle_count_cumulative',
    y='min_track_cycle_voltage',
    color='Group',
    title="Min Track Cycle Voltage vs. Cumulative Cycle Count",
    labels={'track_cycle_count_cumulative': 'Cycle Count', 'min_track_cycle_voltage': 'Voltage (V)'},
    hover_data={"sample_id": True} 
)

# Update traces to include smaller points and lines
fig.update_traces(mode='markers+lines', marker=dict(size=8), line=dict(width=2))

# Customize axes
fig.update_yaxes(
    #range=[2.2, 3.1],
    title='Voltage (V)',
    tickfont=dict(size=22),
    titlefont=dict(size=22),
    mirror=True,
    ticks='outside',
    showline=True,
    linewidth=2,
    linecolor='grey'
)
fig.update_xaxes(
    range=[0, 100],
    title='Cycle Number',
    tickfont=dict(size=22),
    titlefont=dict(size=22),
    mirror=True,
    ticks='outside',
    showline=True,
    linewidth=2,
    linecolor='grey'
)

# Add a horizontal dotted line at 2.45V
fig.add_shape(
    type="line",
    x0=0,
    y0=2.45,
    x1=100,
    y1=2.45,
    line=dict(color="black", width=0.5, dash="dot")
)

# Update layout for a clean appearance
fig.update_layout(
    autosize=False,
    width=900,
    height=600,
    font=dict(size=20),
    plot_bgcolor='white'
)

# Show the plot
fig.show()


### Plot Reliability Survival

In [35]:
# Sort the DataFrame to prioritize rows based on the desired order
MLTrackCycleEach = MLTrackCycleEach.sort_values(
    by=['sample_id', 'track_cycle_count_cumulative', 'TestCycleStart_datetime'], 
    ascending=[True, False, False]
)

def select_cycle_count(group):
    # Check if any of the failure conditions are met and track_cycle_count_cumulative > 0
    failure_conditions = (group[['dvdt_failure', 'min_track_cycle_voltage_failure', 'CapacityChargeFraction_failure']] == 1).any(axis=1)
    failure_rows = group[failure_conditions & (group["track_cycle_count_cumulative"] > 0)]
    
    if not failure_rows.empty:
        selected_row = failure_rows.nsmallest(1, "track_cycle_count_cumulative")  # Keep the first lowest non-zero failure
        selected_row["Shorted"] = True  # Mark as shorted
    else:
        selected_row = group.nlargest(1, "track_cycle_count_cumulative")  # Keep the highest if no failure is found
        selected_row["Shorted"] = False  # Mark as not shorted
    
    return selected_row

MLLastTrackCycle = MLTrackCycleEach.groupby('sample_id', group_keys=False).apply(select_cycle_count)

# Ensure TestCycleEnd_date is in datetime format
MLLastTrackCycle['TestCycleEnd_date'] = pd.to_datetime(MLLastTrackCycle['TestCycleEnd_date'])

# Get today's date in the same format
today_date = pd.to_datetime(datetime.today().date())

# Update 'Shorted' to False if TestCycleEnd_date is today
#MLLastTrackCycle.loc[MLLastTrackCycle['TestCycleEnd_date'] == today_date, 'Shorted'] = False




plotly_colors = [
    # 'rgb(99, 110, 250)',    # Blue
    'rgb(239, 85, 59)',     # Red-orange
    'rgb(0, 204, 150)',     # Green
    'rgb(171, 99, 250)',    # Purple
    'rgb(255, 161, 90)',    # Orange
    'rgb(25, 211, 243)',    # Cyan
    'rgb(255, 102, 146)',   # Pink
    'rgb(182, 232, 128)',   # Light green
    'rgb(255, 151, 255)',   # Light pink
    'rgb(254, 203, 82)'     # Yellow-orange
]*10

# Create figure
fig = make_subplots()
color_list = plotly_colors
fill_color_list = ['rgba' + c[3:-1] + ', 0.15)' for c in color_list]

# Initialize Kaplan-Meier fitter
kmf = KaplanMeierFitter(alpha=0.05)

# Offset for unique x-values to prevent overlapping indices
x_offset = 1e-6
#x_offset = 1

# Iterate over groups in MLLastTrackCycle
for i, (group, group_data) in enumerate(MLLastTrackCycle.groupby('Group')):
    # Prepare data for Kaplan-Meier fitting
    durations = group_data['track_cycle_count_cumulative'].to_numpy()
    event_observed = group_data['Shorted'].to_numpy()

    # Fit Kaplan-Meier model  
    kmf.fit(durations=durations, event_observed=event_observed)

    # Add survival curve trace
    fig.add_trace(
        go.Scatter(
            x=kmf.survival_function_.index + (i * x_offset),  # Add offset to x-values
            y=kmf.survival_function_['KM_estimate'] * 100,
            mode="lines",
            line=dict(shape="hv", width=3, color=color_list[i]),
            name=f"{group} (N={len(group_data)})",
        )
    )

    # Add confidence interval trace
    ci = kmf.confidence_interval_survival_function_
    

    if not ci.empty:
        fig.add_trace(
            go.Scatter(
                x=np.concatenate([ci.index, ci.index[::-1]]) + (i * x_offset),  # Offset x-values
                y=np.concatenate([
                    ci['KM_estimate_upper_0.95'] * 100,
                    (ci['KM_estimate_lower_0.95'] * 100)[::-1]
                ]),
                mode="lines",
                fill="toself",
                fillcolor=fill_color_list[i],
                line=dict(color='rgba(255,255,255,0)'),
                showlegend=False,
            )
        )

    # Add markers for hover labels
    predicted = kmf.predict(durations)
    survival_probabilities = np.array(predicted) if isinstance(predicted, (np.ndarray, pd.Series)) else np.array([predicted])

    customdata = np.stack(
        [
            group_data['sample_id'].to_numpy(),  # Sample ID
            group_data['ReliabilityCycles'].to_numpy(),  # Reliability cycles
            survival_probabilities * 100  # Survival percentage
        ],
        axis=-1
    )

    # Add marker trace for hover
    fig.add_trace(
    go.Scatter(
        x=durations + (i * x_offset),  # Add offset to x-values
        y=survival_probabilities * 100,
        mode="markers",
        marker=dict(size=8, symbol="circle", color=color_list[i]),
        customdata=customdata,
        hovertemplate=(
            "<b>Sample ID:</b> %{customdata[0]}<br>"
            "<b>Reliability Cycles:</b> %{customdata[1]}<br>"
            "<b>Survival %:</b> %{customdata[2]:.2f}%"
        ),
        hoverinfo="x+y+name+text",  # Specify exactly what to show in the hover
        name=f"{group} Markers",  # Unique name for each group
        showlegend=False,  # Prevent marker legends from cluttering
    )
)




# Finalize layout
fig.update_layout(
    title="Survival Plot by Group",
    xaxis_title="Cycle Count",
    yaxis_title="Survival Probability (%)",
    legend_title="Groups",
    font=dict(size=14),
    plot_bgcolor="white",
    width=900,
    height=600
)

fig.add_shape(
            # Line Vertical
            dict(
                type="line",
                x0=60,
                y0=0,
                x1=60,
                y1=105,
                line=dict(
                    color="Grey",
                    width=3,
                    dash="dash",
                ),
            )
        )
fig.add_trace(
    go.Scatter(
        x=[10],
        y=[95],
        mode="markers",
        marker=dict(color="red", symbol="circle", size=20),
        hovertext='95% Survival',
        name="",
        # remove from legend
        showlegend=False,
    ),
    secondary_y=False,
    )
    # adding gate 2 22L target
fig.add_trace(
    go.Scatter(
        x=[60],
        y=[95],
        mode="markers",
        marker=dict(color="blue", symbol="circle", size=20),
        hovertext='95% Survival',
        name="",
        # remove from legend
        showlegend=False,
    ),
    secondary_y=False,
    )

fig.update_yaxes(range=[0, 105], showline=True, linewidth=1, linecolor="black", mirror=True)
fig.update_xaxes(range=[0, 120], showline=True, linewidth=1, linecolor="black", mirror=True)

fig.show()


### Produce ML Tracker Spreadsheet

In [36]:
# Rename columns and assign to df_screening
df_screening = MLScreenMaster[
    ['ML6T_id', 'cell_tier_group', 'stage4_yield', 'TestCycleStart_date', 'ElectricalTestTool', 'ElectricalTestChannel']
].rename(
    columns={
        'ML6T_id': 'MultilayerID',
        'cell_tier_group': 'Tier',
        'stage4_yield': 'ML Screen Yield',
        'TestCycleStart_date': 'Build Date',
        'ElectricalTestTool': 'Tool',
        'ElectricalTestChannel': 'Channel'
    }
)

# Relabel 'false' to 'Pass' and 'true' to 'Failed'
df_screening['ML Screen Yield'] = df_screening['ML Screen Yield'].replace({1: 'Pass', 0: 'Failed'})


# Merge 'MLLastTrackCycle' into 'df_screening'
df_screening = pd.merge(
    df_screening,
    MLLastTrackCycle[['sample_id', 'ElectricalTestTool', 'ElectricalTestChannel', 'Shorted', 'ReliabilityCycles', 'Test_Version']],
    left_on='MultilayerID',  # Column in 'df_screening'
    right_on='sample_id',    # Column in 'MLLastTrackCycle'
    how='left'               # Use 'left' join to keep all rows from 'df_screening'
)

# Overwrite 'Tool' and 'Channel' where 'ElectricalTestTool' and 'ElectricalTestChannel' have values
df_screening['Tool'] = df_screening['ElectricalTestTool'].combine_first(df_screening['Tool'])
df_screening['Channel'] = df_screening['ElectricalTestChannel'].combine_first(df_screening['Channel'])

# Drop the additional columns no longer needed
df_screening = df_screening.drop(columns=['ElectricalTestTool', 'ElectricalTestChannel', 'sample_id'])

# Rename 'CycleFailure' to 'Cycle Status'
df_screening = df_screening.rename(columns={'Shorted': 'Cycle Status'})

# Replace 'True' with 'Shorted' and 'False' with 'Live' in the 'Cycle Status' column
df_screening['Cycle Status'] = df_screening['Cycle Status'].replace({True: 'Shorted', False: 'Live'})

#save the updated dataframe
Output_name = 'ML_TrackingResults.xlsx'
df_screening.to_excel(Output_name, index=False)
